In [1]:
# all codes come are changed based on LDR code.
# 用添加噪声的数据进行训练
import argparse
import torch.optim as optim
import os
import yaml
from mcrgan.default import _C as config
from mcrgan.default import update_config
import torch
from mcrgan.trainer import MCRTrainer
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.models import get_BNN_models
from mcrgan.models import get_noise
from mcrgan.models import get_noise_uniform
from mcrgan.models import DiscriminatorMNIST
from mcrgan.models import weights_init_mnist_model_noise

import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os
from mcrgan.loss import MCRGANloss
import random
import time
random.seed(0)

In [2]:
# 导入Fmnist数据集，进行对比
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
DATASET = 'FMNIST'
ROOT    = 'pth/to/the/dataset'
BATCH_SIZE = 2048
WORKERS = 8
DATA_SAMPLE = 50000
LABEL = 0

dataloader, dataset = get_dataloader(
        data_name=DATASET,
        root=ROOT,
        batch_size=BATCH_SIZE,
        num_workers=WORKERS)

In [3]:
import collections

def get_netG(netD,netV,net_noise,radius=1):
    models = [netD, netV,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    fed_state_dict = collections.OrderedDict()
    for key in weight_keys:
        fed_state_dict[key] = worker_state_dict[0][key] + radius*worker_state_dict[2][key]*torch.log(1+torch.exp(worker_state_dict[1][key]))
    return fed_state_dict

# new netG already optimized by adam and time learning rate, so we only using netG to update netV
def update_netV(netV,netG,net_noise,fed_state_dict,radius=1):
    models =[netV,netG,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    update_dict = collections.OrderedDict()
    for key in weight_keys:
        update_dict[key] = worker_state_dict[0][key] - (worker_state_dict[1][key]-fed_state_dict[key])*(worker_state_dict[2][key]*radius)/torch.log(1+torch.exp(-worker_state_dict[0][key]))
    return update_dict

# generate different distribution via gaussian init and uniform
def get_cauchy_noise(netD,net_noise1,net_noise2):
    models =[netD,net_noise1, net_noise2]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    update_dict = collections.OrderedDict()
    for key in weight_keys:
        update_dict[key] = worker_state_dict[1][key] / worker_state_dict[2][key]
    return update_dict
        

In [4]:
#models
# train hyperparameters, batch norm not used var
#store_Var = []
import time
from torch_mimicry.training import scheduler, logger, metric_log

from test_acc import nearsub
import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features2
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os

start_time = time.time()
#global_step = 0
epsilon_noise = 0.1
num_steps = 1000
n_dis = 1
n_dis2 = 1
numclasses =10
lr1 = 1e-3
lr2 = 1e-3
log_data = metric_log.MetricLog()
data_time = time.time()
iter_dataloader = iter(dataloader)
data, label = next(iter_dataloader)
data_time = time.time() - data_time
real_cpu = data.to(device)
real_label = label.clone().detach()

netD, netG, netV = get_BNN_models(DATASET, device, LABEL)
net_noise = get_noise(DATASET, device, LABEL)
fed_state_dict = get_netG(netD,netV,net_noise)
netG.load_state_dict(fed_state_dict)
optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))

Acc_results_D = []
Acc_results_G = []


transform = transforms.Compose(
            [transforms.Resize(32),
             transforms.ToTensor(),
             transforms.Normalize(0.5, 0.5)])

trainset = datasets.FashionMNIST(root=ROOT, train=True, download=True, transform=transform)
testset = datasets.FashionMNIST(root=ROOT, train=False, download=True, transform=transform)

testloader = torch.utils.data.DataLoader(
        testset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)

trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
n_comp = 10


In [5]:
# golden search algorithm to determined the radius of the sample ball, due to traditional updating netV is too slow
# no meaning for n_dis =2 for our case, just to have output for every update
# sample too much would arise cuda memory error
def gs_radius(radius1,radius2,netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2


def gs_radius2(radius1,radius2, netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise(DATASET, device, LABEL)
    net_noise1 = get_noise(DATASET, device, LABEL)
    net_noise_cauchy = get_cauchy_noise(netD,net_noise,net_noise1)
    net_noise.load_state_dict(net_noise_cauchy)
    #net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2

def gs_radius3(radius1,radius2, netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise_uniform(DATASET, device, LABEL)

    #net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2

In [6]:
#acc at first for NetD
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(testloader, netD, netG)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netD, netG)


print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

#acc at first for NetD
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetD, Acc at first',count/10000)
Acc_results_D.append(count/10000)

#acc at first for NetG
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)


#acc at first
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetG, Acc at first',count/10000)
Acc_results_G.append(count/10000)

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.47s/it]


----------------------------
Train_z, Test_z
PCA: 0.8321000000000001
SVD: 0.8279
NetD, Acc at first 0.8387


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.48s/it]


----------------------------
Train_z, Test_z
PCA: 0.8085
SVD: 0.8073
NetG, Acc at first 0.8437


In [7]:
# data selected
mean_V = []
n_dis2 = 1
n_dis = 1
mode = 1
gam3 = 1
mcr_gan_loss = MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
loss_V = 0
last_V = -1000
num_steps = 500
lr1 = 1e-3
lr2 = 1e-3
optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
meanV_sum = []
radius_list = []
radius = 5

for epoch in range(num_steps):
    if 1==1:
        
        #radius,_ = gs_radius(radius1=0,radius2=100,netD=netD,real_cpu=real_cpu,real_label=real_label,tor=1e-5,n_dis=2,lam=0.618,mode=mode)
        #radius_list.append(radius)
        #print(radius)

        for i in range(n_dis):
            netD.zero_grad()
            optD.zero_grad()
            Z = netD(real_cpu)
            net_noise = get_noise(DATASET, device, LABEL)
            radius,_ = gs_radius(radius1=0,radius2=100,netD=netD,real_cpu=real_cpu,real_label=real_label,tor=1e-5,n_dis=2,lam=0.618,mode=mode)
            radius_list.append(radius)
            print(radius)
            fed_state_dict = get_netG(netD,netV,net_noise,radius)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errD, errD_EC = mcr_gan_loss(Z, Z_bar, real_label, 1, n_dis)
            print(errD, errD_EC)
            errD.backward()
            optD.step()
        temp = 0
        for j in range(n_dis2):
            netG.zero_grad()
            optG.zero_grad()

            Z = netD(real_cpu)
            net_noise = get_noise(DATASET, device, LABEL)
            radius,_ = gs_radius(radius1=0,radius2=100,netD=netD,real_cpu=real_cpu,real_label=real_label,tor=1e-5,n_dis=2,lam=0.618,mode=mode)
            radius_list.append(radius)
            print(radius)
            fed_state_dict = get_netG(netD,netV,net_noise,radius)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG, errG_EC = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            print(errG, errG_EC)
            temp = errG
            errG = (-1) * errG
            errG.backward()
            optG.step()

            update_dic = update_netV(netV,netG,net_noise,fed_state_dict,radius)
            netV.load_state_dict(update_dic)

        loss_V = temp
        

99.97995053415953
tensor(-57.8772, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-6.9267, device='cuda:0', grad_fn=<NegBackward0>), tensor(-6.4033, device='cuda:0', grad_fn=<NegBackward0>), tensor(-44.5472, device='cuda:0', grad_fn=<AddBackward0>)]
99.98259958255836
tensor(-47.3263, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-2.6360, device='cuda:0', grad_fn=<NegBackward0>), tensor(-6.5335, device='cuda:0', grad_fn=<NegBackward0>), tensor(-38.1567, device='cuda:0', grad_fn=<AddBackward0>)]
38.17762457845868
tensor(-46.4718, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-2.6360, device='cuda:0', grad_fn=<NegBackward0>), tensor(-6.4713, device='cuda:0', grad_fn=<NegBackward0>), tensor(-37.3644, device='cuda:0', grad_fn=<AddBackward0>)]
45.12611854059224
tensor(-53.4244, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-4.2714, device='cuda:0', grad_fn=<NegBackward0>), tensor(-6.5349, device='cuda:0', grad_fn=<NegBackward0>), tensor(-42.6181, device='cuda:0', grad_fn=<AddBackw

5.327360729349223
tensor(-98.0182, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-16.9236, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.6986, device='cuda:0', grad_fn=<NegBackward0>), tensor(-72.3960, device='cuda:0', grad_fn=<AddBackward0>)]
5.006369301402145
tensor(-94.8664, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-16.9236, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.7312, device='cuda:0', grad_fn=<NegBackward0>), tensor(-70.2117, device='cuda:0', grad_fn=<AddBackward0>)]
5.630835686550213
tensor(-95.7385, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-16.9205, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.4619, device='cuda:0', grad_fn=<NegBackward0>), tensor(-71.3560, device='cuda:0', grad_fn=<AddBackward0>)]
5.5629919752297985
tensor(-93.9441, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-16.9205, device='cuda:0', grad_fn=<NegBackward0>), tensor(-6.8313, device='cuda:0', grad_fn=<NegBackward0>), tensor(-70.1923, device='cuda:0', grad_fn=<Add

5.940147684709621
tensor(-104.4272, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-21.3604, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.4881, device='cuda:0', grad_fn=<NegBackward0>), tensor(-75.5787, device='cuda:0', grad_fn=<AddBackward0>)]
5.79692075337874
tensor(-104.7449, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-21.5670, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.1915, device='cuda:0', grad_fn=<NegBackward0>), tensor(-75.9864, device='cuda:0', grad_fn=<AddBackward0>)]
5.511172409214584
tensor(-105.4139, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-21.5670, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.0329, device='cuda:0', grad_fn=<NegBackward0>), tensor(-75.8140, device='cuda:0', grad_fn=<AddBackward0>)]
5.090371949510939
tensor(-107.2194, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-22.2795, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.5130, device='cuda:0', grad_fn=<NegBackward0>), tensor(-76.4270, device='cuda:0', grad_fn=<A

6.0161908916378115
tensor(-112.6249, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-24.8207, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.5269, device='cuda:0', grad_fn=<NegBackward0>), tensor(-79.2773, device='cuda:0', grad_fn=<AddBackward0>)]
5.026744272418848
tensor(-111.9637, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-24.8207, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.7415, device='cuda:0', grad_fn=<NegBackward0>), tensor(-78.4014, device='cuda:0', grad_fn=<AddBackward0>)]
5.51311480914238
tensor(-113.1299, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-24.9945, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.7036, device='cuda:0', grad_fn=<NegBackward0>), tensor(-79.4318, device='cuda:0', grad_fn=<AddBackward0>)]
5.4538254614361
tensor(-111.8852, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-24.9945, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.3426, device='cuda:0', grad_fn=<NegBackward0>), tensor(-78.5480, device='cuda:0', grad_fn=<Ad

4.922588476487566
tensor(-116.5906, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-26.9790, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.0804, device='cuda:0', grad_fn=<NegBackward0>), tensor(-80.5312, device='cuda:0', grad_fn=<AddBackward0>)]
5.314077555935773
tensor(-116.8971, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-27.2329, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.8314, device='cuda:0', grad_fn=<NegBackward0>), tensor(-80.8328, device='cuda:0', grad_fn=<AddBackward0>)]
5.245127081878719
tensor(-115.8375, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-27.2329, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.3280, device='cuda:0', grad_fn=<NegBackward0>), tensor(-80.2766, device='cuda:0', grad_fn=<AddBackward0>)]
5.724509576110462
tensor(-116.6334, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-27.4523, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.4008, device='cuda:0', grad_fn=<NegBackward0>), tensor(-80.7802, device='cuda:0', grad_fn=<

...

4.793235483869067
tensor(-135.3829, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.4429, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.5211, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.4188, device='cuda:0', grad_fn=<AddBackward0>)]
4.957836216619003
tensor(-135.5842, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.4393, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.2336, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.9112, device='cuda:0', grad_fn=<AddBackward0>)]
5.608366738243021
tensor(-136.3691, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.4393, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.1512, device='cuda:0', grad_fn=<NegBackward0>), tensor(-89.7785, device='cuda:0', grad_fn=<AddBackward0>)]
4.989109565495021
tensor(-136.1965, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.3937, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.4438, device='cuda:0', grad_fn=<NegBackward0>), tensor(-89.3590, device='cuda:0', grad_fn=<

5.327629593312098
tensor(-134.0727, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.3613, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.1363, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.5751, device='cuda:0', grad_fn=<AddBackward0>)]
5.757236486984832
tensor(-137.3129, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.3613, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.4942, device='cuda:0', grad_fn=<NegBackward0>), tensor(-90.4574, device='cuda:0', grad_fn=<AddBackward0>)]
5.166180042671472
tensor(-137.9666, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.4312, device='cuda:0', grad_fn=<NegBackward0>), tensor(-10.0985, device='cuda:0', grad_fn=<NegBackward0>), tensor(-90.4369, device='cuda:0', grad_fn=<AddBackward0>)]
5.960845210214977
tensor(-134.9575, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.4312, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.8907, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.6356, device='cuda:0', grad_fn=

5.487224985640152
tensor(-134.5918, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.4257, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.5009, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.6652, device='cuda:0', grad_fn=<AddBackward0>)]
6.06206849739311
tensor(-134.8328, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.4104, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.6776, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.7448, device='cuda:0', grad_fn=<AddBackward0>)]
5.288169834763401
tensor(-136.8206, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.4104, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.4450, device='cuda:0', grad_fn=<NegBackward0>), tensor(-89.9652, device='cuda:0', grad_fn=<AddBackward0>)]
5.083542680879661
tensor(-135.7228, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.4360, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.4774, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.8094, device='cuda:0', grad_fn=<A

5.258193107961152
tensor(-135.7544, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.3971, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.4487, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.9085, device='cuda:0', grad_fn=<AddBackward0>)]
5.4073075948354345
tensor(-135.0086, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.3971, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.9727, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.6388, device='cuda:0', grad_fn=<AddBackward0>)]
5.307564381307937
tensor(-134.5028, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.4450, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.8426, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.2153, device='cuda:0', grad_fn=<AddBackward0>)]
5.2443909483937174
tensor(-137.0384, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.4450, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.6651, device='cuda:0', grad_fn=<NegBackward0>), tensor(-89.9283, device='cuda:0', grad_fn

5.025079256437577
tensor(-136.0218, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.4816, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.5995, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.9407, device='cuda:0', grad_fn=<AddBackward0>)]
5.491730735519765
tensor(-137.2248, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.4504, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.6039, device='cuda:0', grad_fn=<NegBackward0>), tensor(-90.1706, device='cuda:0', grad_fn=<AddBackward0>)]
5.920038433575967
tensor(-137.5348, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.4504, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.6320, device='cuda:0', grad_fn=<NegBackward0>), tensor(-90.4524, device='cuda:0', grad_fn=<AddBackward0>)]
4.723963521895609
tensor(-136.3985, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.4921, device='cuda:0', grad_fn=<NegBackward0>), tensor(-10.2800, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.6264, device='cuda:0', grad_fn=

In [8]:
radius_list

[99.97995053415953,
 99.98259958255836,
 38.17762457845868,
 45.12611854059224,
 11.43793767576066,
 7.4561613803678055,
 5.651744884465896,
 6.0562320441025825,
 18.811227921457508,
 5.339640607057247,
 9.42569234552723,
 5.524559640032748,
 5.5390933896905,
 5.535712919144031,
 6.194978688667195,
 6.7593035322381,
 5.194279675900196,
 5.419714952968483,
 5.92560111216801,
 5.021340337331054,
 5.29766558296617,
 4.579915966544212,
 4.728011751906983,
 6.075847280967209,
 5.989891751674763,
 5.469297119900499,
 5.191530879340439,
 5.278238916001288,
 99.92626396700042,
 4.717367314193273,
 6.282688520883852,
 5.937026846694925,
 5.541128170664217,
 5.327360729349223,
 5.006369301402145,
 5.630835686550213,
 5.5629919752297985,
 5.7466347502540085,
 5.82157238967904,
 5.424273863560556,
 6.098370319631853,
 7.429778714111981,
 5.534866281083704,
 4.84606752203199,
 4.7249161827727,
 5.536708281758468,
 5.2278284283927094,
 6.442818788659792,
 5.806946722415053,
 5.764857170050041,
 4.84

In [9]:
#acc at first for NetD
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(testloader, netD, netG)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netD, netG)


print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

#acc at first for NetD
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetD, Acc at first',count/10000)
Acc_results_D.append(count/10000)

#acc at first for NetG
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)


#acc at first
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetG, Acc at first',count/10000)
Acc_results_G.append(count/10000)


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.60s/it]


----------------------------
Train_z, Test_z
PCA: 0.8069999999999999
SVD: 0.8011
NetD, Acc at first 0.8582


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.51s/it]


----------------------------
Train_z, Test_z
PCA: 0.8072
SVD: 0.8034
NetG, Acc at first 0.8385


In [10]:
torch.save(netD, 'FMNIST_NetD_changing2-mode1.pth')
torch.save(netV, 'FMNIST_NetV_changing2-mode1.pth')
torch.save(netG, 'FMNIST_NetG_changing2-mode1.pth')

#netD = torch.load('FMNIST_NetD.pth')
#netV = torch.load('FMNIST_NetV.pth')
#netG = torch.load('FMNIST_NetG.pth')

#mode = 1
#gam3 = 1

In [11]:
r_list = [0,0.1,0.2,0.5,1,2,3,4,6,8,10]
sample_num = 20

In [12]:
# sample 20 times of NetG at final radius, NetD is fixed, length 0.1,0.2,0.5,1,2,4,6,8,10,15,20
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample = []
for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[1])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample.append(count/10000)
    
print(max(Acc_results_G_sample),min(Acc_results_G_sample),np.mean(Acc_results_G_sample),np.var(Acc_results_G_sample))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.55s/it]


----------------------------
Train_z, Test_z
PCA: 0.7946
SVD: 0.7896
NetG_fixD, Acc at sample 1 0.8537


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.54s/it]


----------------------------
Train_z, Test_z
PCA: 0.7957
SVD: 0.7897000000000001
NetG_fixD, Acc at sample 2 0.8536


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.52s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.7964
SVD: 0.7886
NetG_fixD, Acc at sample 18 0.855


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.55s/it]


----------------------------
Train_z, Test_z
PCA: 0.796
SVD: 0.7892
NetG_fixD, Acc at sample 19 0.8534


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.55s/it]


----------------------------
Train_z, Test_z
PCA: 0.7957
SVD: 0.7898000000000001
NetG_fixD, Acc at sample 20 0.8553
0.8553 0.8526 0.8542099999999999 3.6789999999999016e-07


In [13]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample2 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[2])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample2.append(count/10000)
    
print(max(Acc_results_G_sample2),min(Acc_results_G_sample2),np.mean(Acc_results_G_sample2),np.var(Acc_results_G_sample2))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.57s/it]


----------------------------
Train_z, Test_z
PCA: 0.7951
SVD: 0.7891
NetG_fixD, Acc at sample 1 0.8537


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.54s/it]


----------------------------
Train_z, Test_z
PCA: 0.7952
SVD: 0.7894
NetG_fixD, Acc at sample 2 0.8537


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.58s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.7948999999999999
SVD: 0.7888
NetG_fixD, Acc at sample 18 0.8538


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.54s/it]


----------------------------
Train_z, Test_z
PCA: 0.7948
SVD: 0.7886
NetG_fixD, Acc at sample 19 0.8524


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.54s/it]


----------------------------
Train_z, Test_z
PCA: 0.794
SVD: 0.7888999999999999
NetG_fixD, Acc at sample 20 0.8529
0.8555 0.8519 0.853455 6.41475000000004e-07


In [14]:
# sample 50 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample3 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[3])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample3.append(count/10000)
    
print(max(Acc_results_G_sample3),min(Acc_results_G_sample3),np.mean(Acc_results_G_sample3),np.var(Acc_results_G_sample3))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.52s/it]


----------------------------
Train_z, Test_z
PCA: 0.7952
SVD: 0.7855
NetG_fixD, Acc at sample 1 0.8512


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.56s/it]


----------------------------
Train_z, Test_z
PCA: 0.7962
SVD: 0.7917
NetG_fixD, Acc at sample 2 0.8539


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.53s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.7959
SVD: 0.7928999999999999
NetG_fixD, Acc at sample 18 0.8498


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.54s/it]


----------------------------
Train_z, Test_z
PCA: 0.7918000000000001
SVD: 0.7862
NetG_fixD, Acc at sample 19 0.8524


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.54s/it]


----------------------------
Train_z, Test_z
PCA: 0.79
SVD: 0.7884
NetG_fixD, Acc at sample 20 0.8517
0.8547 0.8498 0.8520049999999999 1.742475000000007e-06


In [15]:
# sample 50 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample4 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[4])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample4.append(count/10000)
    
print(max(Acc_results_G_sample4),min(Acc_results_G_sample4),np.mean(Acc_results_G_sample4),np.var(Acc_results_G_sample4))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.53s/it]


----------------------------
Train_z, Test_z
PCA: 0.7986
SVD: 0.7923
NetG_fixD, Acc at sample 1 0.8502


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.56s/it]


----------------------------
Train_z, Test_z
PCA: 0.7969999999999999
SVD: 0.792
NetG_fixD, Acc at sample 2 0.8488


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.53s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.7971
SVD: 0.7937
NetG_fixD, Acc at sample 18 0.8483


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.55s/it]


----------------------------
Train_z, Test_z
PCA: 0.7948
SVD: 0.7928999999999999
NetG_fixD, Acc at sample 19 0.8482


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.55s/it]


----------------------------
Train_z, Test_z
PCA: 0.7935
SVD: 0.7885
NetG_fixD, Acc at sample 20 0.8479
0.851 0.8466 0.8488149999999999 1.1182749999999889e-06


In [16]:
# sample 50 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample5 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[5])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample5.append(count/10000)
    
print(max(Acc_results_G_sample5),min(Acc_results_G_sample5),np.mean(Acc_results_G_sample5),np.var(Acc_results_G_sample5))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.54s/it]


----------------------------
Train_z, Test_z
PCA: 0.7981
SVD: 0.7951
NetG_fixD, Acc at sample 1 0.8281


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.54s/it]


----------------------------
Train_z, Test_z
PCA: 0.8012
SVD: 0.7963
NetG_fixD, Acc at sample 2 0.8406


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.55s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.8002
SVD: 0.7997
NetG_fixD, Acc at sample 18 0.8295


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.53s/it]


----------------------------
Train_z, Test_z
PCA: 0.7986
SVD: 0.7933
NetG_fixD, Acc at sample 19 0.8323


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.57s/it]


----------------------------
Train_z, Test_z
PCA: 0.8065
SVD: 0.7995
NetG_fixD, Acc at sample 20 0.8302
0.8406 0.8246 0.832475 1.7741875000000016e-05


In [17]:
# sample 50 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample6 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[6])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample6.append(count/10000)
    
print(max(Acc_results_G_sample6),min(Acc_results_G_sample6),np.mean(Acc_results_G_sample6),np.var(Acc_results_G_sample6))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.53s/it]


----------------------------
Train_z, Test_z
PCA: 0.8076
SVD: 0.8041
NetG_fixD, Acc at sample 1 0.7484


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.54s/it]


----------------------------
Train_z, Test_z
PCA: 0.8071
SVD: 0.7984
NetG_fixD, Acc at sample 2 0.77


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.55s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.8026
SVD: 0.7986
NetG_fixD, Acc at sample 18 0.7841


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.50s/it]


----------------------------
Train_z, Test_z
PCA: 0.8033
SVD: 0.7947
NetG_fixD, Acc at sample 19 0.7248


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.55s/it]


----------------------------
Train_z, Test_z
PCA: 0.8039000000000001
SVD: 0.8
NetG_fixD, Acc at sample 20 0.651
0.7841 0.5275 0.69513 0.0047940291000000005


In [18]:
# sample 50 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample7 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[7])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample7.append(count/10000)
    
print(max(Acc_results_G_sample7),min(Acc_results_G_sample7),np.mean(Acc_results_G_sample7),np.var(Acc_results_G_sample7))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.54s/it]


----------------------------
Train_z, Test_z
PCA: 0.81
SVD: 0.8055
NetG_fixD, Acc at sample 1 0.3059


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.54s/it]


----------------------------
Train_z, Test_z
PCA: 0.8089999999999999
SVD: 0.8046
NetG_fixD, Acc at sample 2 0.2838


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.54s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.8093
SVD: 0.8017
NetG_fixD, Acc at sample 18 0.428


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.55s/it]


----------------------------
Train_z, Test_z
PCA: 0.8065
SVD: 0.8031
NetG_fixD, Acc at sample 19 0.3868


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.49s/it]


----------------------------
Train_z, Test_z
PCA: 0.8073
SVD: 0.8065
NetG_fixD, Acc at sample 20 0.4223
0.428 0.2495 0.34016500000000005 0.0039101402750000005


In [19]:
# sample 50 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample8 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[8])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample8.append(count/10000)
    
print(max(Acc_results_G_sample8),min(Acc_results_G_sample8),np.mean(Acc_results_G_sample8),np.var(Acc_results_G_sample8))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.55s/it]


----------------------------
Train_z, Test_z
PCA: 0.8095
SVD: 0.806
NetG_fixD, Acc at sample 1 0.1509


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.55s/it]


----------------------------
Train_z, Test_z
PCA: 0.8069999999999999
SVD: 0.8069999999999999
NetG_fixD, Acc at sample 2 0.1325


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.56s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.8122
SVD: 0.813
NetG_fixD, Acc at sample 18 0.1143


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.55s/it]


----------------------------
Train_z, Test_z
PCA: 0.8096
SVD: 0.813
NetG_fixD, Acc at sample 19 0.1334


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.54s/it]


----------------------------
Train_z, Test_z
PCA: 0.8159
SVD: 0.8110999999999999
NetG_fixD, Acc at sample 20 0.14
0.1747 0.0992 0.13414 0.0004344214000000001


In [20]:
# sample 50 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample9 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[9])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample9.append(count/10000)
    
print(max(Acc_results_G_sample9),min(Acc_results_G_sample9),np.mean(Acc_results_G_sample9),np.var(Acc_results_G_sample9))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.58s/it]


----------------------------
Train_z, Test_z
PCA: 0.8108
SVD: 0.8058
NetG_fixD, Acc at sample 1 0.1158


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.55s/it]


----------------------------
Train_z, Test_z
PCA: 0.8023
SVD: 0.8015
NetG_fixD, Acc at sample 2 0.101


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.56s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.8089
SVD: 0.8102
NetG_fixD, Acc at sample 18 0.1391


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.51s/it]


----------------------------
Train_z, Test_z
PCA: 0.8049999999999999
SVD: 0.8056
NetG_fixD, Acc at sample 19 0.1078


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.54s/it]


----------------------------
Train_z, Test_z
PCA: 0.8102
SVD: 0.8084
NetG_fixD, Acc at sample 20 0.1048
0.1391 0.0865 0.107555 0.00016356047500000004


In [21]:
# sample 50 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample10 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[10])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample10.append(count/10000)
    
print(max(Acc_results_G_sample10),min(Acc_results_G_sample10),np.mean(Acc_results_G_sample10),np.var(Acc_results_G_sample10))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.55s/it]


----------------------------
Train_z, Test_z
PCA: 0.8143
SVD: 0.8091
NetG_fixD, Acc at sample 1 0.1077


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.54s/it]


----------------------------
Train_z, Test_z
PCA: 0.8013
SVD: 0.8016
NetG_fixD, Acc at sample 2 0.0848


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.52s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.8134
SVD: 0.8097
NetG_fixD, Acc at sample 18 0.1054


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.53s/it]


----------------------------
Train_z, Test_z
PCA: 0.8114
SVD: 0.8085
NetG_fixD, Acc at sample 19 0.0956


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.55s/it]


----------------------------
Train_z, Test_z
PCA: 0.812
SVD: 0.8071
NetG_fixD, Acc at sample 20 0.1329
0.1387 0.0823 0.10290999999999999 0.0002444129
